# Form Parsing using Google Cloud Document AI

This notebook shows how to use Google Cloud Document AI to parse a campaign disclosure form.

It accompanies this Medium article:
https://medium.com/@lakshmanok/how-to-parse-forms-using-google-cloud-document-ai-68ad47e1c0ed

In [ ]:
!sudo chown -R jupyter:jupyter /home/jupyter/imported/formparsing.ipynb

In [1]:
from IPython.display import Markdown as md

### change to reflect your notebook
_nb_repo = 'training-data-analyst'
_nb_loc = "blogs/form_parser/formparsing.ipynb"
_nb_title = "Form Parsing Using Google Cloud Document AI"

### no need to change any of this
_nb_safeloc = _nb_loc.replace('/', '%2F')
_nb_safetitle = _nb_title.replace(' ', '+')
md("""
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name={1}&url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2F{3}%2Fblob%2Fmaster%2F{2}&download_url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2F{3}%2Fraw%2Fmaster%2F{2}">
    <img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/> Run in AI Platform Notebook</a>
  </td>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/{3}/blob/master/{0}">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/GoogleCloudPlatform/{3}/blob/master/{0}">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/GoogleCloudPlatform/{3}/master/{0}">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>
""".format(_nb_loc, _nb_safetitle, _nb_safeloc, _nb_repo))


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name=Form+Parsing+Using+Google+Cloud+Document+AI&url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Ftraining-data-analyst%2Fblob%2Fmaster%2Fblogs%2Fform_parser%2Fformparsing.ipynb&download_url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Ftraining-data-analyst%2Fraw%2Fmaster%2Fblogs%2Fform_parser%2Fformparsing.ipynb">
    <img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/> Run in AI Platform Notebook</a>
  </td>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/blogs/form_parser/formparsing.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/blogs/form_parser/formparsing.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/GoogleCloudPlatform/training-data-analyst/master/blogs/form_parser/formparsing.ipynb">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>


### Document

As an example, let's take this US election campaign disclosure form.

In [2]:
%%bash
if [ ! -f scott_walker.pdf ]; then
   curl -O https://storage.googleapis.com/practical-ml-vision-book/images/scott_walker.pdf
fi

In [3]:
!ls *.pdf

scott_walker.pdf


In [4]:
from IPython.display import IFrame
IFrame("./scott_walker.pdf", width=600, height=300)

**Note**: If the file is not visible, simply open the PDF file by double-clicking on it in the left hand menu.

## Upload to Cloud Storage

Document AI works with documents on Cloud Storage, so let's upload the doc.

In [ ]:
BUCKET="ai-analytics-solutions-kfpdemo"  # CHANGE to a bucket that you own

In [ ]:
!gsutil cp scott_walker.pdf gs://{BUCKET}/formparsing/scott_walker.pdf

Copying file://scott_walker.pdf [Content-Type=application/pdf]...
/ [1 files][209.7 KiB/209.7 KiB]                                                
Operation completed over 1 objects/209.7 KiB.                                    


In [ ]:
!gsutil ls gs://{BUCKET}/formparsing/scott_walker.pdf

gs://ai-analytics-solutions-kfpdemo/formparsing/scott_walker.pdf


## Enable Document AI

1. First enable Document AI in your project by visiting
https://console.developers.google.com/apis/api/documentai.googleapis.com/overview

2. Find out who you are running as:

In [ ]:
!gcloud auth list

                  Credentialed Accounts
ACTIVE  ACCOUNT
*       379218021631-compute@developer.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



3. Create a service account authorization by visiting
https://console.cloud.google.com/iam-admin/serviceaccounts/create
Give this service account Document AI Core Service Account authorization

4. Give the above ACTIVE ACCOUNT the ability to use the service account you just created.

## Call Document AI

In [ ]:
%%bash
PDF="gs://ai-analytics-solutions-kfpdemo/formparsing/scott_walker.pdf" # CHANGE to your PDF file
REGION="us"  # change to EU if the bucket is in the EU

cat <<EOM > request.json
{
   "inputConfig":{
      "gcsSource":{
         "uri":"${PDF}"
      },
      "mimeType":"application/pdf"
   },
   "documentType":"general",
   "formExtractionParams":{
      "enabled":true
   }
}
EOM

# Send request to Document AI.
PROJECT=$(gcloud config get-value project)
echo "Sending the following request to Document AI in ${PROJECT} ($REGION region), saving to response.json"
cat request.json

curl -X POST \
  -H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
  -H "Content-Type: application/json; charset=utf-8" \
  -d @request.json \
  https://${REGION}-documentai.googleapis.com/v1beta2/projects/${PROJECT}/locations/us/documents:process \
  > response.json

Sending the following request to Document AI in ai-analytics-solutions (us region), saving to response.json
{
   "inputConfig":{
      "gcsSource":{
         "uri":"gs://ai-analytics-solutions-kfpdemo/formparsing/scott_walker.pdf"
      },
      "mimeType":"application/pdf"
   },
   "documentType":"general",
   "formExtractionParams":{
      "enabled":true
   }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3366k    0 3366k  100   246   832k     60  0:00:04  0:00:04 --:--:--  832k


In [ ]:
!tail response.json

                }
              ]
            },
            "orientation": "PAGE_UP"
          }
        }
      ]
    }
  ]
}


**Note**: If you get a 403 PERMISSION DENIED error, please re-run all the cells from the top.

## Parse the response

Let's use Python to parse the response and pull out specific fields.


In [ ]:
import json
ifp = open('response.json')
response = json.load(ifp)

In [ ]:
allText = response['text']
print(allText[:100])

10/09/2020 15:51
Image# 202010099285076251
PAGE 1/14
7
REPORT OF RECEIPTSFECAND DISBURSEMENTSFORM 3P


### Option 1: Parsing blocks of text

As an example, let's try to get the "Cash on Hand". This is in Page 2 and the answer is $75,931.36
All the data in the document is the allText field. we just need to find the right starting and ending index
for what we want to extract.

In [ ]:
print(allText.index("CASH ON HAND"))

1719


We know that "Cash on Hand" is on Page 2.

In [ ]:
response['pages'][1]['blocks'][5]

{'layout': {'textAnchor': {'textSegments': [{'startIndex': '1716',
     'endIndex': '1827'}]},
  'confidence': 1,
  'boundingPoly': {'normalizedVertices': [{'x': 0.068627454, 'y': 0.24873738},
    {'x': 0.6764706, 'y': 0.24873738},
    {'x': 0.6764706, 'y': 0.25757575},
    {'x': 0.068627454, 'y': 0.25757575}]},
  'orientation': 'PAGE_UP'}}

In [ ]:
response['pages'][1]['blocks'][5]['layout']['textAnchor']['textSegments'][0]

{'startIndex': '1716', 'endIndex': '1827'}

In [ ]:
startIndex = int(response['pages'][1]['blocks'][5]['layout']['textAnchor']['textSegments'][0]['startIndex'])
endIndex = int(response['pages'][1]['blocks'][5]['layout']['textAnchor']['textSegments'][0]['endIndex'])
allText[startIndex:endIndex]

'6. CASH ON HAND AT BEGINNING OF REPORTING PERIOD .............................................................\n'

Cool, we are at the right part of the document! Let's get the next block, which should be the actual amount.

In [ ]:
def extractText(allText, elem):
    startIndex = int(elem['textAnchor']['textSegments'][0]['startIndex'])
    endIndex = int(elem['textAnchor']['textSegments'][0]['endIndex'])
    return allText[startIndex:endIndex].strip()

amount = float(extractText(allText, response['pages'][1]['blocks'][6]['layout']))
print(amount)

75931.36


### Option 2: Parsing form fields

What we did with blocks of text was quite low-level. Document AI understands that forms tend to have key-value pairs, and part of the JSON response includes these extracted key-value pairs as well.

Besides FormField Document AI also supports getting Paragraph and Table from the document.

In [ ]:
response['pages'][1].keys()

dict_keys(['pageNumber', 'dimension', 'layout', 'blocks', 'paragraphs', 'lines', 'tokens', 'tables', 'formFields'])

In [ ]:
response['pages'][1]['formFields'][2]

{'fieldName': {'textAnchor': {'textSegments': [{'startIndex': '1719',
     'endIndex': '1765'}]},
  'confidence': 0.9962783,
  'boundingPoly': {'normalizedVertices': [{'x': 0.0922335, 'y': 0.24873738},
    {'x': 0.4584429, 'y': 0.24873738},
    {'x': 0.4584429, 'y': 0.2587827},
    {'x': 0.0922335, 'y': 0.2587827}]},
  'orientation': 'PAGE_UP'},
 'fieldValue': {'textAnchor': {'textSegments': [{'startIndex': '1716',
     'endIndex': '1842'}]},
  'confidence': 0.9962783,
  'boundingPoly': {'normalizedVertices': [{'x': 0.068627454, 'y': 0.24873738},
    {'x': 0.90849674, 'y': 0.24873738},
    {'x': 0.90849674, 'y': 0.26767677},
    {'x': 0.068627454, 'y': 0.26767677}]},
  'orientation': 'PAGE_UP'}}

In [ ]:
fieldName = extractText(allText, response['pages'][1]['formFields'][2]['fieldName'])
fieldValue = extractText(allText, response['pages'][1]['formFields'][2]['fieldValue'])
print('key={}\nvalue={}'.format(fieldName, fieldValue))

key=CASH ON HAND AT BEGINNING OF REPORTING PERIOD
value=6. CASH ON HAND AT BEGINNING OF REPORTING PERIOD .............................................................
75931.36
, , .


Enjoy!

Copyright 2021 Google Inc. Licensed under the Apache License, Version 2.0 (the \"License\"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License